In [1]:
import os
import cv2
import numpy as np
import pandas as pd

from tqdm import tqdm

import color_manipulation as cm

from HSV_manipulations import get_HSV_RGB

In [2]:
def mean_hue_from_hsv_linear_saturation(hsv_img: np.ndarray) -> float:
    """
    Computes the circular mean hue of an HSV image (OpenCV format),
    weighted linearly by saturation (S channel).

    Hue channel must be in [0,179].
    Saturation channel is [0,255].

    Returns
    -------
    float
        Weighted mean hue in degrees (0–360).
    """

    # Extract H and S channels
    hue = hsv_img[:, :, 0].astype(np.float32)
    sat = hsv_img[:, :, 1].astype(np.float32) / 255.0  # normalize to [0,1]

    # Convert hue to radians (0–2π)
    rad = np.deg2rad(hue * 2.0)

    # Linear weight: more saturated pixels contribute more
    weights = sat

    # Avoid division by zero
    if np.sum(weights) == 0:
        return np.nan  # or fallback

    # Weighted circular mean
    mean_x = np.sum(weights * np.cos(rad)) / np.sum(weights)
    mean_y = np.sum(weights * np.sin(rad)) / np.sum(weights)

    mean_angle = np.arctan2(mean_y, mean_x)
    mean_deg = np.rad2deg(mean_angle)

    # Normalize to [0,360)
    if mean_deg < 0:
        mean_deg += 360

    return mean_deg

       


In [3]:
STEP_SIZE = 1/2
ORANGE_TEAL = 30.0

input_dir = 'in/examples'
output_dir = 'out/examples_step_size_1_2_sat'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

img_files = []
for filename in os.listdir(input_dir):
    img_files.append(filename)

In [4]:
for img_file in tqdm(img_files):

    img_file_no_ext = os.path.splitext(img_file)[0]
    if not os.path.exists(os.path.join(output_dir, img_file_no_ext)):
        os.makedirs(os.path.join(output_dir, img_file_no_ext))

    bgr_img = cv2.imread(os.path.join(input_dir, img_file))
    rgb_img = bgr_img[:, :, ::-1]
    hsv_img, img_rgb = get_HSV_RGB(os.path.join(input_dir, img_file), scale_factor=1)

    # Saving original
    cv2.imwrite(os.path.join(output_dir, img_file_no_ext, f'{img_file_no_ext}_original.png'), bgr_img)

    # Saving plots
    mean_hue = mean_hue_from_hsv_linear_saturation(hsv_img)
    cm.plot_color_wheel_and_mean_angle(hsv_img, os.path.join(output_dir, img_file_no_ext, f'{img_file_no_ext}_color_wheel.png'), mean_hue)

    # L1C1
    l1c1_angle = cm.compute_angle(ORANGE_TEAL, mean_hue, c=1, l=1, l_float=STEP_SIZE)
    cm.recolor_image(hsv_img, l1c1_angle, 1, os.path.join(output_dir, img_file_no_ext, f'{img_file_no_ext}_L1C1.png'))

    # L2C1
    l2c1_angle = cm.compute_angle(ORANGE_TEAL, mean_hue, c=1, l=2, l_float=STEP_SIZE)
    cm.recolor_image(hsv_img, l2c1_angle, 1, os.path.join(output_dir, img_file_no_ext, f'{img_file_no_ext}_L2C1.png'))

    # L1C2
    l1c2_angle = cm.compute_angle(ORANGE_TEAL, mean_hue, c=2, l=1, l_float=STEP_SIZE)
    cm.recolor_image(hsv_img, l1c2_angle, 1, os.path.join(output_dir, img_file_no_ext, f'{img_file_no_ext}_L1C2.png'))

    # L2C2
    l2c2_angle = cm.compute_angle(ORANGE_TEAL, mean_hue, c=2, l=2, l_float=STEP_SIZE)
    cm.recolor_image(hsv_img, l2c2_angle, 1, os.path.join(output_dir, img_file_no_ext, f'{img_file_no_ext}_L2C2.png'))

    # Comparrison
    cm.display_original_and_variants(
        os.path.join(output_dir, img_file_no_ext), 
        os.path.join(output_dir, img_file_no_ext, f'{img_file_no_ext}_comparison.png'))

    # Data
    configurations = ['orange-teal', 'mean hue', 'L1C1', 'L2C1', 'L1C2', 'L2C2']
    angles = [ORANGE_TEAL, mean_hue, l1c1_angle, l2c1_angle, l1c2_angle, l2c2_angle]

    df = pd.DataFrame({
        "configuration": configurations,
        "angle": angles
    })

    df.to_csv(os.path.join(output_dir, img_file_no_ext, 'angles.csv'), index=False)


100%|██████████| 10/10 [10:19<00:00, 61.95s/it]
